<a href="https://colab.research.google.com/github/aaalexlit/medium_articles/blob/main/Leveraging_Huggingface_with_pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup
## Install required libraries and load extension

In [1]:
%%capture
!pip install transformers
!pip install datasets
!pip3 install memory_profiler
%load_ext memory_profiler

## Check if GPU is available and set everything to run on it if it's the case

In [4]:
import torch

if torch.cuda.is_available():
  device = 0
  torch_device = torch.device('cuda')
else:
  device = -1
  torch_device = torch.device('cpu')

# Inference using original code

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("amandakonet/climatebert-fact-checking")
tokenizer = AutoTokenizer.from_pretrained("amandakonet/climatebert-fact-checking")

In [6]:
sample_claim = ['Beginning in 2005, however, polar ice modestly receded for several years']
sample_evidence = ['Polar Discovery "Continued Sea Ice Decline in 2005']

In [7]:
def predict_using_sample_code(claims, evidences):
  features = tokenizer(claims,
                       evidences,
                       padding='max_length', 
                       truncation=True, 
                       return_tensors="pt", 
                       max_length=512)
  # if GPU is available the code will use it
  features.to(torch_device)
  model.to(torch_device)

  model.eval()
  with torch.no_grad():
    scores = model(**features).logits
    label_mapping = ['entailment', 'contradiction', 'neutral']
    labels = [label_mapping[score_max] for score_max in scores.argmax(dim=1)]
    return labels

## Run inference on a provided sample

In [8]:
%%time
%memit predict_using_sample_code(sample_claim, sample_evidence)

peak memory: 1400.44 MiB, increment: 11.71 MiB
CPU times: user 1.24 s, sys: 140 ms, total: 1.38 s
Wall time: 1.7 s


## Load more samples

In [9]:
from datasets import load_dataset
cf_df = load_dataset("amandakonet/climate_fever_adopted", split='test').to_pandas()
input_claims = cf_df['claim'].values.tolist()
input_evidences = cf_df['evidence'].values.tolist()

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/4298 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/1842 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/amandakonet___parquet/amandakonet--climate_fever_adopted-5a2eed1d355d5a34/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


In [10]:
len(input_claims), len(input_evidences)

(1535, 1535)

The following cell normally kills the notebook with Out Of Memory  
but that one time I ran it I got loads of RAM in the environment for some reason

In [ ]:
%%time
%memit labels = predict_using_sample_code(input_claims, input_evidences)

peak memory: 70648.12 MiB, increment: 52950.69 MiB
CPU times: user 12min 46s, sys: 6min 37s, total: 19min 23s
Wall time: 3min 48s


# Use `transformers.pipeline` instead

In [13]:
from transformers import pipeline

def predict_using_pipelines(claims: [str], evidences: [str]) -> ([str], [float]):
    def claim_evidence_pair_data():
        for claim, evidence in zip(claims, evidences):
            yield {"text": claim, "text_pair": evidence}

    pipe = pipeline("text-classification",
                    model=model,
                    tokenizer=tokenizer,
                    truncation=True, 
                    padding=True, 
                    device=device)
    labels, probs = [], []
    for out in pipe(claim_evidence_pair_data(), batch_size=1):
        labels.append(out['label'])
        probs.append(out['score'])
    return labels, probs

In [14]:
%%time
%memit pred_labels, pred_probs = predict_using_pipelines(input_claims, input_evidences)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


peak memory: 1434.24 MiB, increment: 9.80 MiB
CPU times: user 4min, sys: 557 ms, total: 4min
Wall time: 4min 7s
